In [ ]:
!pip install tensorflow==2.2.0

In [ ]:
from zipfile import ZipFile
with ZipFile('/content/drive/MyDrive/Computer_Vision_Project/Pneumonia_Image_Classification/DATA/pneumonia_dataset.zip') as file:
  file.extractall('/content/Data/')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATASET_PATH = '/content/Data/'

normal_images = pd.DataFrame({'Image_Path':glob(DATASET_PATH + "pneumonia_dataset/train/normal/*.png"), 'label':'No'})
pneumonia_images = pd.DataFrame({'Image_Path':glob(DATASET_PATH + "pneumonia_dataset/train/pneumonia/*.png"), 'label':'Yes'})

train_df = pd.concat([normal_images,pneumonia_images], axis=0, sort =False)
test_df = pd.read_csv(DATASET_PATH + "pneumonia_dataset/test.csv")
test_df['Image_Path'] = "/content/Data/pneumonia_dataset/test/" + test['filename']

print(train_df.shape) # (2425,2)
print(test_df.shape) # (606,1)

train_df , validation_df = train_test_split(train_df,random_state=101 ,test_size=0.20)

train_df.to_csv(DATASET_PATH + 'train_df.csv', index=False)
test_df.to_csv(DATASET_PATH + 'test_df.csv', index=False)
validation_df.to_csv(DATASET_PATH + 'validation_df.csv', index=False)

(2425, 2)
(606, 2)


In [ ]:
BATCH_SIZE = 32
EPOCHS = 20
IMAGE_SHAPE = (224,224,3)
TARGET_SIZE = (224,224)

train_data = pd.read_csv(DATASET_PATH + 'train_df.csv')
validation_data = pd.read_csv(DATASET_PATH + 'validation_df.csv')

# Training Generator
train_datagen = ImageDataGenerator(rescale=1./255 , rotation_range=20 , width_shift_range=0.10, height_shift_range=0.10,
                                   horizontal_flip=True, vertical_flip=True,zoom_range=0.10, shear_range=0.10, fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(dataframe=train_data,x_col='Image_Path', y_col='label',target_size=TARGET_SIZE,
                                                    class_mode='categorical',shuffle=True,batch_size=BATCH_SIZE)

# Validation datagen
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = train_datagen.flow_from_dataframe(dataframe=validation_data, x_col='Image_Path', y_col='label',target_size=TARGET_SIZE,
                                                    class_mode='categorical',shuffle=False,batch_size=BATCH_SIZE)


STEPS_PER_EPOCHS = len(train_data) // BATCH_SIZE
VAL_STEPS_PER_EPOCHS = len(validation_data) // BATCH_SIZE

# Show Images with Train DataGenerator
sample_image = train_data.sample(1).reset_index(drop=True)
sample_generator = train_datagen.flow_from_dataframe(sample_image,x_col='Image_Path', y_col='label',target_size=TARGET_SIZE,
                                                    class_mode='categorical',shuffle=False,batch_size=BATCH_SIZE)

plt.figure(figsize=(18,12))
for i in np.arange(16):
    plt.subplot(4,4,i+1)
    for image, label in sample_generator:
        plt.imshow(image[0])
        plt.axis('off')
        break

In [ ]:
from sklearn.metrics import precision_score , recall_score , f1_score , log_loss, accuracy_score
from tensorflow.keras.callbacks import Callback

class Validation(Callback):
    def __init__(self, data_generator,df):
        self.validation_generator = data_generator
        self.dataframe = df

    def on_epoch_end(self, epoch, logs=None):
        prediction = self.model.predict_generator(self.validation_generator)
        prediction_prob = prediction[:,1]
        prediction = np.argmax(prediction, axis=1)
        actual = self.dataframe['label'].tolist()
        actual = [1 if x =='Yes' else 0 for x in actual]
        recall = recall_score(actual, prediction, pos_label=0)
        precision = precision_score(actual, prediction, pos_label=0)
        accuracy = accuracy_score(actual, prediction)
        # F1_score = f1_score(actual, prediction)
        loss = log_loss(actual, prediction_prob)

        logs['val_recall'] ,logs['val_precision'],logs['val_accuracy'],logs['val_loss'] = recall , precision, accuracy, loss

val_df =pd.read_csv(DATASET_PATH + "validation_df.csv")
validation = Validation(validation_generator, val_df)

In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, MaxPooling2D , GlobalAveragePooling2D , Dropout
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping , ReduceLROnPlateau , Callback
from tensorflow.keras.optimizers import Adam , SGD

pre_trained_model = MobileNet(input_shape=IMAGE_SHAPE , include_top=False, weights='imagenet')

for layer in pre_trained_model.layers:
    layer.trainable = True

last_output_layer = pre_trained_model.layers[-1].output
x = GlobalAveragePooling2D()(last_output_layer)
x = Dense(512, activation='relu')(x)
x = Dropout(0.20)(x)
x = Dense(256,activation='relu')(x)
x = Dropout(0.20)(x)
x = Dense(128,activation='relu')(x)
x = Dropout(0.20)(x)
x = Dense(2,activation='softmax')(x)

model = Model(inputs = pre_trained_model.input , outputs = x)
model.summary()

model.compile(optimizer=Adam(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])

model_checkpoint = ModelCheckpoint(filepath="/content/drive/MyDrive/Computer_Vision_Project/Pneumonia_Image_Classification/DATA/{val_accuracy:.2f}{epoch:2d}.h5",
                                   monitor='val_accuracy',
                                   save_weights_only=False,
                                   save_best_only=False,
                                   mode='max',
                                   verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.1,
                              patience=10,
                              verbose=1,
                              mode ='min')

model.fit_generator(generator=train_generator, steps_per_epoch=STEPS_PER_EPOCHS ,callbacks=[validation,model_checkpoint,reduce_lr],
                     epochs=60, verbose=1)
model.save(DATASET_PATH + "model.h5")

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [ ]:
from tensorflow.keras.models import load_model

test_df = pd.read_csv(DATASET_PATH + "test_df.csv")
model = load_model('/content/drive/MyDrive/Computer_Vision_Project/Pneumonia_Image_Classification/DATA/0.7450.h5') #77.61

sample_submission = pd.read_csv('/content/Data/pneumonia_dataset/sample_submission.csv')

# Test Generator
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(dataframe=test_df, x_col='Image_Path',y_col=None, target_size=TARGET_SIZE,shuffle=False, batch_size=BATCH_SIZE,
                                                class_mode=None)

prediction = model.predict_generator(test_generator)
sample_submission['label'] = np.argmax(prediction, axis=1)
labels = train_generator.class_indices # No-0 and Yes-1
sample_submission['label'] = sample_submission['label'].replace({1:'pneumonia',0:'normal'})
sample_submission.to_csv(DATASET_PATH + 'sample_submission_6.csv', index=False)
print("Sample_Submission File Saved..")

Found 606 validated image filenames.
Sample_Submission File Saved..
